In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('max_rows', 20)

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 3)
plt.rcParams['font.family'] = 'sans-serif'

In [ ]:
AQI_FILEPATH = os.path.join(os.curdir, 'data', 'aqi.csv')

df = pd.read_csv(filepath_or_buffer=AQI_FILEPATH, parse_dates=['datetime'], index_col='datetime', na_values='NR')
df_bak = df.copy()

# 保留降雨量為 NR 值的資料
df_withnr = pd.read_csv(filepath_or_buffer=AQI_FILEPATH, parse_dates=['datetime'], index_col='datetime')
df_withnr_bak = df_withnr.copy()

# 保留還未將 datetime 設為 index 的資料
df_noindex = pd.read_csv(filepath_or_buffer=AQI_FILEPATH)
df_noindex_bak = df_noindex.copy()

# 讀取未處理過的原始資料
AQI_ORIG_FILEPATH = os.path.join(os.curdir, 'data', 'aqi_original.csv')

df_orig = pd.read_csv(AQI_ORIG_FILEPATH)
df_orig_bak = df_orig.copy()

# 讀取還未使用 pivot_table() 處理過的資料
AQI_NOPIVOT_FILEPATH = os.path.join(os.curdir, 'data', 'aqi_nopivot.csv')

df_nopivot = pd.read_csv(AQI_NOPIVOT_FILEPATH, na_values=['NR'])
df_nopivot.replace(r'[-]?\D*[.]?\D*[#*x]+', np.nan, regex=True, inplace=True)
df_nopivot.loc[:, 'value'] = df_nopivot.loc[:, 'value'].astype(np.float64)
df_nopivot_bak = df_orig.copy()

In [ ]:
df.head()

# 第 6 章：修改資料

有時候僅僅執行運算或是篩選是不夠的。我們常常聽到「清理資料」的名詞，意思就是在分析以前，資料常常要經過大規模的整理，才能夠被順利地使用。

這個章節將會介紹許多涵蓋整份資料結構改變的操作，例如轉換資料型別、轉置、或是將部分資料結合再運算、以及清除遺漏的值、也可能要進行補值。

希望經過這些處理以後，可以讓各位要進行分析的數據更完整、也更準確！

## 修正資料

### 利用 `.loc` 和 `.iloc` 在特定範圍修改資料

`.loc`, `.iloc` attribute 除了可以取得指定範圍的資料之外，也可以用來編輯該範圍的資料內容。

編輯多筆資料時，請注意資料的形狀是否與指定的範圍相同；而如果只對她們指定單一數值，將會把該範圍的數值都用單一數值取代。

In [ ]:
# 觀察降雨量 (column label: 'RAINFALL') 中，無降雨量資料的情況
df.loc[:, 'RAINFALL'].head(10)

In [ ]:
# 例：利用先前介紹的 isna() 來為遺漏值補值
rainfall_isna = df.loc[:, 'RAINFALL'].isna()
df.loc[rainfall_isna, :] = 0
display(df.loc[:, 'RAINFALL'].head(10))

# 當然，這了例子使用這種做法不夠直覺
# 我們之後還會講到 `.fillna()` method，更好用！

In [ ]:
# 還原資料
df = df_bak.copy()

### `.replace()`：替換數值

`.replace()` method 的用法，除了單純將固定的數值或文字替換掉以外，更進階且快速的用法則可以使用正規表示式來建立取代的條件，礙於篇幅無法介紹，請自行參閱正規表示式的相關文章。

* 常用 Parameters：
    1. `inplace`：是否選擇直接修改資料而不回傳處理結果，預設為 `False`
* 參考文件：
    * [pandas.DataFrame.replace](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html)
    * [pandas.Series.replace](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.replace.html)
    * [Wikipedia | 正規表示式](https://zh.wikipedia.org/zh-tw/正規表示式)


In [ ]:
# 觀察降雨量 (column label: 'RAINFALL') 中，
# 無降雨量資料 (NR) 的情況
df_withnr.loc[:, 'RAINFALL'].head(10)

In [ ]:
# 將降雨量數值為 NR 的取代成 0
df_withnr.replace({'RAINFALL': {'NR': 0}}, inplace=True)
display(df_withnr.loc[:, 'RAINFALL'].head(10))

In [ ]:
# 還原資料
df_withnr = df_withnr_bak.copy()

### `.rename()`：編輯 Row 或 Column 的 Label

如果 label 不易辨識或是不如預期時，之後操作時出錯的機率可能會上升，像是取錯 column 放入其他函式的問題，造成結果也是錯的。為了避免這種情況發生，事前將 label 準備成容易一眼辨識的名稱也是很重要的。

取代規則的格式可以是一個 `dict` 物件，他們的 key 跟 value 則分別是**被取代的 label** 跟**取代之後的 label**；也可以放入一個 function。

參考文件：
* [pandas.DataFrame.rename](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html#pandas.DataFrame.rename)
* [pandas.Series.rename](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.rename.html)
    

In [ ]:
# 示範使用 dict (Dictionary) 逐一修改
# 例：將 columns label 為 'AMB_TEMP' 以及 'WIND_SPEED'
# 改成小寫英文字母，且將 '_' 改成 ' '
df.rename(columns={'AMB_TEMP':'amb temp', 'WIND_SPEED':'wind speed'}, inplace=True)
display(df.columns)

In [ ]:
# 還原資料
df = df_bak.copy()

In [ ]:
# 示範使用 lambda function
# 例：將所有 columns label 改成小寫英文字母，且將 '_' 改成 ' '
df.rename(columns=lambda x: x.lower().replace('_', ' '), inplace=True)
display(df.columns)

In [ ]:
# 還原資料
df = df_bak.copy()

### `.sort_values()`：以數值大小排序

適當的將資料給排序也有助於觀察資料的樣態。這裡先介紹的是依照數值大小來排序的 method，往後還會介紹到針對 index (row index) 來排序的 method。

* 常用 Parameters（以 DataFrame 為例）：
    1. `axis`：操作的座標軸
        * `0`：將 dataframe 以 row 方向排序
        * `1`：將 dataframe 以 column 方向排序
    1. `by`：設定排序參照的對象
        * `str`：以某一個 columns label 或 row label 排序
        * `list`：依照順序以多個 columns label 或 row label 排序
    1. `ascending`：是否使用升冪排序，預設為 `True`
    1. `na_position`：資料中有 `NaN` 數值時，指定排序後要將 `NaN` 放置於開頭或結尾，預設為 `last`
        * `first`：把 NaN 放在開頭
        * `last`：把 NaN 放在最後
    1. `inplace`：是否選擇直接修改資料而不回傳處理結果，預設為 `False`
* 參考文件：
    * [pandas.DataFrame.sort_values](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html)
    * [pandas.Series.sort_values](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.sort_values.html)


In [ ]:
# 將資料以一氧化碳 (columns label: 'CO') 由小到大排序，且 NaN 放在開頭
df.sort_values(axis=0, by='CO', ascending=True, na_position='first', inplace=True)
display(df.loc[:, 'CO'].tail(10))

In [ ]:
# 還原資料
df = df_bak.copy()

In [ ]:
# Series 也可以調用 sort_values()
df.loc[:, 'CO'].sort_values(ascending=False).head(10)

### `.sort_index()`：以 Index Label 大小排序

比較常見的情況在於使用 `datetime` 物件當作 row index 時，可以讓資料依照時間順序來排列。

* 常用 Parameters : 
    1. `axis`：操作的座標軸
        * `0`：將 dataframe 以 row 方向排序
        * `1`：將 dataframe 以 column 方向排序
    1. `ascending`：是否使用升冪排序，預設為 `True`
    1. `inplace`：是否選擇直接修改資料而不回傳處理結果，預設為 `False`
* 參考文件：
    * [pandas.DataFrame.sort_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_index.html)
    * [pandas.Series.sort_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.sort_values.html)


In [ ]:
# 將資料以 row index 降冪排序
df.sort_index(ascending=False, inplace=True)
display(df.head())

In [ ]:
# 還原資料
df = df_bak.copy()

In [ ]:
# 將 columns index 升冪（字母 A 到 Z 排序）
df.sort_index(axis=1, inplace=True, ascending=True)
display(df.head())

In [ ]:
# 還原資料
df = df_bak.copy()

### `.reset_index()`：重設 Index Label 

預設會將 row index 給重設為從 0 開始的數列，在常見的使用情境在資料列有修改的情況，例如用 `.drop()` method 刪除部分的 row 以後。

* 常用 Parameters：
    1. `drop`：重設 index 之後是否刪除被取代的 index
        * `True`：刪除原始的 row index
        * `False`：保留原始的 row index
    1. `inplace`：是否選擇直接修改資料而不回傳處理結果，預設為 `False`
* 參考文件：
    * [pandas.DataFrame.reset_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html)
    * [pandas.Series.reset_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.reset_index.html)

In [ ]:
# 觀察一下原本的 index label
df.head()

In [ ]:
# 重設 index label 且保留原本的 index label 作為一個 column
df.reset_index(drop=False).head()

In [ ]:
# 重設 index label 且"不"保留原本的 index label
df.reset_index(drop=True).head()

### `.set_index()`：將某個 Column (Column Labels) 設為 Index (Row Labels)

調用 `set_index()` method 以將目前的 index (row labels) 用某個 column 的內容 (column labels) 來取代，常見的情境可以將帶有 `datetime` 物件的 column 給設為 index。

參考文件：[pandas.DataFrame.set_index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.set_index.html)

    

In [ ]:
# set_index
# 將 日期 設定為 index
df_noindex.set_index('datetime', inplace=True)
display(df_noindex.head())

In [ ]:
# 還原資料
df_noindex = df_noindex_bak.copy()

### `.drop()`：刪除特定的 row 或 column


調用 `drop()` method 可以用來刪除指定的 row/column。

使用時，可以在 parameter `labels` 指定要刪除的 index (row label) 或是 column (column label)，或是個別在 parameter `index` 或 `columns` 指定要刪除的 label。

* 文件連結：
    * [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html)
    * [pandas.Series.drop](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.drop.html)



In [ ]:
# 先使用 describe 觀察非甲烷碳氫化合物 (column label: 'NMHC') 的資料分佈
df['NMHC'].describe()

In [ ]:
# 例：刪除非甲烷碳氫化合物為負值的資料
# 使用 boolean index 取得非甲烷碳氫化合物 為 0 的 index (row label)
nmhc_iszero_indexes = df.loc[df['NMHC'] == 0, :].index

# 使用 drop 將這些 index (row label) 的資料刪除
# 再觀察資料分佈
df.drop(index=nmhc_iszero_indexes)['NMHC'].describe()

In [ ]:
# 還原資料
df = df_bak.copy()

### `.dropna()`：刪除遺漏值 (NA)

將資料中帶有遺漏值的部分給刪除。

* 常用 Parameters：
    1. `axis`：操作的座標軸
        * `0`：將 dataframe 以 row 方向排序
        * `1`：將 dataframe 以 column 方向排序
    1. `how`：刪除資料的策略，預設為 `any`
        * `all`：若此 column 或 row 的每一個數值全部都是 `NaN`，才刪除此 column 或 index
        * `any`：若此 column 或 row 的只要有任何數值為 `NaN`，就刪除此 column  或 index
* 參考文件：
    * [pandas.DataFrame.dropna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html)
    * [pandas.Series.dropna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.dropna.html)

In [ ]:
# 先製作一個比較小的 DataFrame 來展示功能
tmp_dropna = pd.DataFrame({"A" : [10, 10, 11, np.nan], 
                           "B" : [5.5, 5.5, np.nan, 10.2], 
                           "C" : [np.nan, 200, 230, 260],
                           "D" : [10, 5.5, 11.0, 10.2]})
tmp_dropna

In [ ]:
# 例：若 row 出現 NaN，則刪除此筆資料
tmp_dropna.dropna(axis=0, how='any')

In [ ]:
# 若 column 出現 NaN 則刪除此筆資料
tmp_dropna.dropna(axis=1, how='any')

### `.fillna()`：為遺漏值補上數值

`.fillna()` method 可以為資料提供一個統一的策略，產生數值來替代遺漏值。
    

* 常用 Parameters：
    1. `value`：給定一個值（可以是 Series，Dataframe，dict）去填補遺失值，例如 `0`
    1. `method`：設定補值策略
        * `bfill` / `backfill`：由後往前補值
        * `pad` / `ffill`：由前往後補值
    1. `inplace`：是否選擇直接修改資料而不回傳處理結果，預設為 `False`
* 參考文件：
    * [pandas.DataFrame.fillna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html)
    * [pandas.Series.fillna](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.fillna.html)
    * [10 MInutes to pandas | Missing Data](https://pandas.pydata.org/pandas-docs/stable/10min.html#missing-data)
    * [Working with missing data | Cleaning / filling missing data | Filling missing values: fillna](https://pandas.pydata.org/pandas-docs/stable/missing_data.html#filling-missing-values-fillna)


> 備註—有關遺漏值的兩三事：
>
> 所謂的**遺漏值，在文件中常稱為 NA (Not available)**，包含 `None`  或  `NaN` (`numpy.NaN`)，
>
> 但是 `' '`（空白字元）或 `numpy.inf`（無限大數）則不屬於 **NA**。

In [ ]:
# 先製作一個比較小的 DataFrame 來展示功能
tmp_fillna = pd.DataFrame({'A' : pd.Series([1, np.nan, 2, np.nan, np.nan, 3]),
                           'B' : pd.Series([np.nan, np.nan, 200, np.nan, np.nan, 300])})
tmp_fillna

In [ ]:
# 補值方法 1: 設定 value 為特定值
# 例：以該 column 的平均值來補值
tmp_fillna.fillna(tmp_fillna.mean())

In [ ]:
# 補值方法 2: bfill（由後往前補值）
tmp_fillna.fillna(method='bfill')

In [ ]:
# 補值方法 3: ffill（由前往後補值）
# .fillna(method='ffill')
tmp_fillna.fillna(method='ffill')

### `.T`：DataFrame 轉置

回傳以對角線為對稱方向對調後的資料。

* 文件連結：[pandas.DataFrame.transpose](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.transpose.html)


In [ ]:
# transpose

# 在 dataframe 資料顯示上，會將 觀察項目 作為 columns，而每一筆觀察資料 作為 rows
# 但是當拿到一份資料時，可能資料紀錄的方式剛好相反，如下，觀察項目在 rows，每個時間點的觀察資料為 columns
# 這時候可以使用 .T 進行轉置

In [ ]:
# 先製作一個比較小的 DataFrame 來展示功能
test_t = pd.DataFrame({'2017-01-01 01:00:00':[20.0, 2.0, 0.20, 0.04], 
                       '2017-01-01 02:00:00':[20.0, 2.2, 0.19, 0.05], 
                       '2017-01-01 03:00:00':[19.0, 2.2, 0.24, 0.08]},
                       index=['AMB_TEMP', 'CH4', 'CO', 'NMHC'])
test_t

In [ ]:
# 嘗試轉置
test_t.T

### `.pivot_table()`：樞紐分析表

樞紐分析表是一種「將一個表格的資料彙總到另一個表格」的分析方法，並對彙總的資料做結合、排序、加總等等操作。

因為資料庫管理系統 (Database Management System, DBMS) 的設計關係，有時候在取出來的資料中，會發現應該置於 column label 的資料會被放在 column values 之內，這時候也可以用樞紐分析表來製作我們較易於用來分析的資料格式。

參考文件：

* [pandas.DataFrame.pivot_table](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot_table.html)
* [pandas.pivot_table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html)
* [10 Minutes to pandas | Reshaping | Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/10min.html#pivot-tables)
* [Reshaping and Pivot Tables | Pivot tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html#pivot-tables)
* [Microsoft Office 支援 | 建立樞紐分析表來分析工作表的資料](https://support.office.com/zh-tw/article/a9a84538-bfe9-40a9-a8e9-f99134456576)


In [ ]:
# 觀察一下尚未做樞紐分析表之前的資料
df_nopivot.head()

In [ ]:
# 製作樞紐分析表
df_pivot = df_nopivot.pivot_table(index='datetime', columns='item', values='value')
# 上面的操作與以下等價
# pd.pivot_table(df_nopivot, index='datetime', columns='item', values='value')

df_pivot.head()

In [ ]:
# 同場加映：將樞紐分析表給還原，用 `melt()`
# 因為程序有點複雜，應用的場景看起來也不多，這裡先保留不講解

# 請自行執行看看，並測試如果不把 column index 的 name 給刪除，會是什麼情況？
del(df_pivot.columns.name)

# 把 row index 給還原回 column
df_pivot.reset_index(inplace=True)

# 「融化」她！
df_pivot.melt(id_vars='datetime', var_name='item', value_name='value').head()
# 上面的操作與以下等價
# pd.melt(df_pivot, id_vars='datetime', var_name='item', value_name='value').head()

### `.apply()`：套用函式

DataFrame 的資料將會依照指定的 Axis 方向套用函式，Series 則會將所有資料套用函式，最後回傳使用公式轉換後的資料。

常見的公式可以直接套用 [Numpy](https://docs.scipy.org/doc/numpy/index.html, [Scipy](http://scipy.org/) 套件提供數學及統計學類的 function，例如：

* [`numpy.sum()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html#numpy.sum)：總和
* [`numpy.mean()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html#numpy.mean)：算數平均數
* [`numpy.median()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.median.html#numpy.median)：中位數
* [`scipy.stats.mode()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html#scipy.stats.mode)：眾數
* [`numpy.percentile()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.percentile.html#numpy.percentile)：四分位數
* [`numpy.var()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.var.html#numpy.var)：變異數
* [`numpy.std()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.std.html#numpy.std)：標準差
* [`numpy.amax()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.amax.html#numpy.amax)：最大值
* [`numpy.amin()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.amin.html#numpy.amin)：最小值

也可以使用 Python 內建的 Function，或是自訂 Function 來轉換資料。

參考文件：

* [pandas.DataFrame.apply](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html)
* [Numpy | Mathematical functions](https://docs.scipy.org/doc/numpy/reference/routines.math.html)
* [Numpy | Statistics](https://docs.scipy.org/doc/numpy/reference/routines.statistics.html)
* [Scipy | Statistical functions (scipy.stats)](https://docs.scipy.org/doc/scipy/reference/stats.html)

In [ ]:
# 計算所有空氣品質資料監測項目的最大值
df.apply(np.amax)
# 與 df.apply(np.amax, axis=0) 等價
# 與 df.apply(np.amax, axis='index') 等價

In [ ]:
# 自訂 Function
def max_minus_min(x):
    return x.max() - x.min()
# 與 max_minus_min = lambda x: x.max() - x.min() 等價

# 計算大氣溫度、甲烷、一氧化碳（column label：'AMB_TEMP', 'CH4', 'CO'）濃度的最大值與最小值的差
df.loc[:, ['AMB_TEMP', 'CH4', 'CO']].apply(max_minus_min)
# 與 df.loc[:, ['AMB_TEMP', 'CH4', 'CO']].apply(lambda x: x.max() - x.min()) 等價

#### 補充：apply(), applymap(), map()

雖然 `apply()` 也可以使用在 Series 上面，但是跟套用於 DataFrame 上的動作卻略有不同：對 DataFrame 調用 `.apply()` method 時，可以選擇將 Column 或是 Row 給輸入 function 做運算；而對 Series 調用時，則會將每一個元素給一一置入 function。

所以建議在處理不同的問題時，根據情況分別使用 `apply()`, `applymap()`, `map()` Methods。

補充比較這三個常見的 Method 的用途：

* `apply()`：常用於 DataFrame 上，將以 Row 或 Column 為單位，將資料分割為許多一維向量，再套用 Function
* `applymap()`：常用於 DataFrame 上，為每一個元素套用 Function
* `map()`：常用於 Series 上，為每一個元素套用 Function（與 `pandas.Series.apply()` 等價）

參考文件：[Difference between map, applymap and apply methods in Pandas
](https://stackoverflow.com/a/19798528/2975670)

In [ ]:
# 自訂 Function
# 例：自訂一個將所有數值 +2 的 Function
plus_two = lambda x: x+2

# 將所有一氧化氮濃度（Column 名稱：'NO'）數值 +2
s_amb_temp = df.loc[:, 'AMB_TEMP']

s_amb_temp.map(plus_two).head()
# 以上操作與以下等價
# s.apply(plus_two) 

而若要將 Series 當作一維向量直接套用 Function 計算的話，可以考慮調用 `pandas.Series.values` Arrtibute 取出 `numpy.ndarray` 物件，再將物件置入 Function 內計算。

In [ ]:
# 例：計算一氧化氮濃度（column label：'NO'）的最小值
# Function 使用 Numpy.amin()
np.amin(s_amb_temp.dropna().values)  # 有些 Function 不支援輸入 NaN 值，
                            # 保險起見，可以使用 dropna() Method 預先排除掉

## 轉換特定 Data Type 的資料

對於 `str` 或是 `datetime` 類型的資料，Pandas 提供了簡易的 method 或 attribute，可以在調用之後擷取出部分的資訊，例如對 `str` 類型的資料做轉換大小寫、分割、取代，對 `datetime` 類型的資料則可以取出年份、月份、日期等資料。

### `.str`：調用特定 Function 轉換 `str` Type 的資料

對 `str` type 的資料調用 `.str` attribute 之後，可以緊接著調用他們的 method 或 attribute，來做一些常用的轉換。舉例如下：

 * `.str.upper()`：將全部英文字母轉換成大寫
 * `.str.lower()` :  將全部英文字母轉換成小寫
 * `.str.replace()`：將固定的數值或文字替換掉以外
 * `.str[ : ]`：取得一部分的文字 (就是 Slice)
 * `.str.len()`：回傳文字長度
 * `.str.startswith()`：檢查文字的開頭是否與文字相符
 * `.str.contains()`：檢查否包含文字
 
參考文件：
 
* [Working with Text Data](https://pandas.pydata.org/pandas-docs/stable/text.html)
* [Series | String handling](https://pandas.pydata.org/pandas-docs/stable/api.html#string-handling)

In [ ]:
# 檢查 'datetime' column 的 date type
df_noindex.dtypes['datetime']

In [ ]:
# 取得 'datetime' column 的年份，也就是 str[:4]
df_noindex.loc[:, 'datetime'].str[:4].head()

In [ ]:
# 取得 'datetime' column 的月份，也就是 str[5:7]
df_noindex.loc[:, 'datetime'].str[5:7].head()

### `.dt`：調用特定 Attribute 以取得 `datetime` Type 物件的資訊

對 `datetime` type 的資料調用 `.dt` attribute 之後，可以緊接著調用他們的 attribute，來取得一些常用的資訊。舉例如下：

* `datetime.year`：回傳年份
* `datetime.month`：回傳月份
* `datetime.day`：回傳日期
* `datetime.hour`：回傳小時
* `datetime.minute`：回傳分鐘
* `datetime.second`：回傳秒數
* `datetime.weekday`：回傳星期幾（依序為星期一 = `0`、星期二 = `1`⋯⋯星期天 = `6`）

In [ ]:
# 先將 df_noindex 的 'datetime' column 的 data type 轉換為 datetime
df_noindex.loc[:, 'datetime'] = pd.to_datetime(df_noindex.loc[:, 'datetime'])
df_noindex.dtypes['datetime']
# 如果直接查詢 df_noindex.dtypes['datetime'] 會得到 '<M8[ns]' 的結果
# 其實也是 datetime
# 請參見：https://stackoverflow.com/a/29218694/2975670

In [ ]:
# 取得 'datetime' column 的小時
df_noindex['datetime'].dt.hour.head()

In [ ]:
# 取得 'datetime' column 的星期幾
df_noindex['datetime'].dt.weekday.head()

In [ ]:
# 還原資料
df_noindex = df_noindex_bak.copy()

## 對資料做結合運算

許多時候我們會把有共同 column 的資料先結合起來，再對他們做一些運算，例如：將許多相同月份的數據集合起來取平均值。這時候就會調用 `.groupby()` method，以節省運算的時間。

相關的操作在[關聯式資料庫 (Relational Database, RDB)](https://zh.wikipedia.org/zh-tw/關聯式資料庫) 上也是很常見的。

參考文件：

* [10 Minuste to pandas | Grouping](https://pandas.pydata.org/pandas-docs/stable/10min.html#grouping)
* [Group By: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby)

In [ ]:
# 讓我們把這個小節會用到的 column：年份、月份、給做出來。
# 先觀察一下 index (row index)
df.index

In [ ]:
# Row index 的型態是 datetime，所以可以直接調用 datetime 的 year, month......等屬性
# 就利用這個特性來做出新的 column
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['hour'] = df.index.hour
df['minute'] = df.index.minute
df['second'] = df.index.second
df['weekday'] = df.index.weekday

df.columns

In [ ]:
df.head()

In [ ]:
pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']])

### `.groupby()`：將具有相同 Column 的資料結合

結合資料後會回傳一個 [`GroupBy` 物件](https://pandas.pydata.org/pandas-docs/stable/api.html#groupby)，可以直接接著呼叫她的 method 來做處理。

* 參考文件：
    * [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)
    * [pandas.Series.groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.groupby.html)

#### 調用 `GroupBy` 物件的 method

可以執行的 method 主要分為 [Indexing, iteration](https://pandas.pydata.org/pandas-docs/stable/api.html#id38)、[Function application](https://pandas.pydata.org/pandas-docs/stable/api.html#function-application) 和 [Computations / Descriptive Stats](https://pandas.pydata.org/pandas-docs/stable/api.html#id39)，請自行參考  API References 文件。

這裡只舉幾個例子。像是用 `.mean()` 來計算平均數：

參考文件：[pandas.core.groupby.GroupBy.mean](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.mean.html)

In [ ]:
# 例：觀察同年、月份的各個空氣品質指標的平均數
df.groupby(['year', 'month']).mean()

另一個例子：使用 `.count()` : 計數。在做計數時，遺漏值 NA 將不列入計數對象。

參考文件：

* [pandas.core.groupby.GroupBy.count](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.count.html)

In [ ]:
# 例：觀察同年、月份的各項空氣品質指標，各有多少筆資料（排除遺漏值）
display(df.groupby(['year', 'month']).count())

#### `.resample()`：同場加映——重新抽樣：不用 `groupby()` 也可以對日期分群

當我們需要轉換**時間序列資料**的頻率跟重新抽樣的時候，就是一個方便的工具。先用 `resample()` method 指定重新抽樣的方式，再搭配  `.apply()` 指定轉換的 function。

在使用時，Series 或 DataFrame 的 Index 需為 **`datetime` 型態的時間序列資料**。

* 參考文件
    * [Time Series / Date functionality | Resampling](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#resampling)
    * [pandas.Series.resample](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.resample.html)
    * [pandas.DataFrame.resample](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html)

In [ ]:
# 對 PM2.5 的資料（column label：'PM2.5'）重新抽樣，再計算出各月 PM 2.5 濃度的平均數
s_pm25 = df.loc[:, 'PM2.5']
s_pm25.resample('M').apply(np.median)

In [ ]:
# 畫出各月 PM 2.5 濃度的平均數折線圖
s_pm25.resample('M').apply(np.median).plot()

若 DataFrame 的 index 不為 `datetime` 型態的時間序列資料，可以使用 parameter `on=` 來指定 `datetime` 型態時間序列資料的 Column 名稱。

In [ ]:
# 先將 df_noindex 的 'datetime' column 的 data type 轉換為 datetime
df_noindex['datetime'] = pd.to_datetime(df_noindex.loc[:, 'datetime'])

# 對 PM2.5、PM10 （column label：'PM10'）的資料重新抽樣，再計算出各月 PM 2.5 濃度的算數平均數
df_orig_pm25_pm10 = df_noindex.loc[:, ['datetime', 'PM2.5', 'PM10']]
df_orig_pm25_pm10.resample('M', on='datetime').apply(np.mean)

# 還原資料
df_noindex = df_noindex_bak.copy()

### `.aggregate()` : 對結合後的資料調用多個敘述統計 function

如果要對一份資料同時製作多個敘述性統計分析，除了調用 `.drescibe()` method 以外，當還有額外的需求時，也可以利用這個 method 來達成。

* 參考文件：
    * [pandas.DataFrame.aggregate](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.aggregate.html)
    * [GroupBy: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/groupby.html#aggregation)

In [ ]:
# 僅使用單一敘述統計的 function
# 例：計算各項空氣品質指摽的平均
display(df.aggregate('mean'))

In [ ]:
# 針對各個 column 指定執行不同的敘述統計 function
# 作法：寫在 dict 內
# 例：計算大氣溫度 (column label: 'AMB_TEMP') 的最小值、
# PM 2.5 (column label: 'PM2.5') 的平均值
# 以及一氧化氮 (column label: 'NO') 的最大值
df.aggregate({'AMB_TEMP':'min', 'PM2.5':'mean', 'NO': 'max'})

In [ ]:
# 對所有 column 指定執行多種不同的敘述統計 function
# 作法：寫在 list 內
# 力：觀察所有空氣品質指標的平均、最大值、最小值、標準差
df.aggregate(['mean', 'max', 'min', 'std'])

In [ ]:
# 也可以配合 groupby() 使用，預先集合相同條件的資料再執行敘述統計 function
# 例：觀察同年、月份資料的平均值、標準差
df.groupby(['year', 'month']).aggregate(['mean', 'std'])